In [ ]:
import pandas as pd
import numpy as np
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import platform
import gc
import sys
import argparse
from glob import glob
from google.colab import drive
from tqdm import tqdm
from pathlib import Path
from joblib import Parallel, delayed
import re
import random
import requests
import urllib.request
import json
from copy import deepcopy
import copy
from dataclasses import dataclass
from tqdm import tqdm
tqdm.pandas()

from konlpy.tag import Mecab
import transformers
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, TrainingArguments, AutoModelForCausalLM, BitsAndBytesConfig, PreTrainedTokenizerFast
from datasets import load_dataset
from trl import DPOTrainer, SFTTrainer
import bitsandbytes as bnb
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model, PeftModel
import logging
logger = logging.getLogger(__name__)
from typing import Optional, Dict, Sequence
from Korpora import Korpora
from Korpora import KowikiTextKorpus, KorNLIKorpus
# from googletrans import Translator
from dask import bag, diagnostics

import torch
import pytorch_lightning as pl
from torch.utils.data import DataLoader
from torch.utils.data import Dataset

In [ ]:
def print_system_specs():
    # Check if CUDA is available
    is_cuda_available = torch.cuda.is_available()
    print("CUDA Available:", is_cuda_available)
# Get the number of available CUDA devices
    num_cuda_devices = torch.cuda.device_count()
    print("Number of CUDA devices:", num_cuda_devices)
    if is_cuda_available:
        for i in range(num_cuda_devices):
            # Get CUDA device properties
            device = torch.device('cuda', i)
            print(f"--- CUDA Device {i} ---")
            print("Name:", torch.cuda.get_device_name(i))
            print("Compute Capability:", torch.cuda.get_device_capability(i))
            print("Total Memory:", torch.cuda.get_device_properties(i).total_memory, "bytes")
    # Get CPU information
    print("--- CPU Information ---")
    print("Processor:", platform.processor())
    print("System:", platform.system(), platform.release())
    print("Python Version:", platform.python_version())
print_system_specs()

In [ ]:
!nvidia-smi

In [ ]:
drive.mount('/content/drive')

In [ ]:
# Load to Data
data_location = '/content/drive/MyDrive/llm'
data_path = Path(data_location)

train = pd.read_csv(data_path / 'train.csv')

In [ ]:
# translate-back_translate

def back_translate(sentence, lang, PROB = 1):
  #languages = ['ko', 'en', 'fr', 'th', 'tr', 'ur', 'ru', 'bg', 'de', 'ar', 'zh-cn', 'hi', 'sw', 'vi', 'es', 'el', 'ja']

  translator = Translator()

  org_lang = translator.detect(sentence).lang # ko

  #random_language = np.random.choice([lang for lang in languages if lang is not org_lang])

  if lang is not org_lang:

    translated = translator.translate(sentence, dest = lang).text
    translated_back = translator.translate(translated, dest = org_lang).text

  else:

    translated_back = sentence

  # if np.random.uniform(0, 1) <= PROB:
  #   output_sentence = translated_back
  # else:
  #   output_sentence = sentence

  return translated_back

# parallel apply

def back_translate_parallel1(dataset, translate_column, lang, save_file = True):
  translate_bag = bag.from_sequence(dataset[translate_column].tolist()).map(lambda x: back_translate_papago(x, lang = lang))

  with diagnostics.ProgressBar():
    bag_completed = translate_bag.compute()

  dataset[f'{translate_column}_tranlsate'] = bag_completed

  return dataset

def back_translate_parallel2(dataset, translate_column, lang, save_file = True):
  try:
    dataset[f'{translate_column}_translate'] = dataset[f'{translate_column}'].progress_apply(lambda x: back_translate_papago(x, 'en'))

  except Exception as e:
    print(f"Error occurred: {str(e)}")
    if save_file:
      save_path = '/content/back_translate.csv'
      if not os.path.exists(f'{save_path}'):
        dataset.to_csv(f'{save_path}', index = False, mode = 'w', encoding = 'utf-8-sig', header = False)
      else:
        dataset.to_csv(f'{save_path}', index = False, mode = 'a', encoding = 'utf-8-sig', header = False)

  return dataset

In [ ]:
# import googletrans
# googletrans.LANGUAGES

train_aug = back_translate_parallel2(dataset = train, translate_column = '답변_1', lang = 'en')